In [1]:
import cv2
import numpy as np
import os
import natsort

GREEN = (0, 255, 0)
Margin = 10

def canny(img, min, max):
    canny_img = cv2.Canny(img, min, max)
    return canny_img

def sobel(img):
    dx = cv2.Sobel(img, cv2.CV_32F, 1, 0) 
    dy = cv2.Sobel(img, cv2.CV_32F, 0, 1)
    mag = cv2.magnitude(dx, dy)
    mag = np.clip(mag, 0, 255).astype(np.uint8) 

    dst = np.zeros(img.shape[:2], np.uint8) 
    dst[mag > 120] = 255 
    return dst

def find_min_max(cnt):
    
    x_min, x_max, y_min, y_max = np.ndarray.min(cnt[...,0]), np.ndarray.max(cnt[...,0]), np.ndarray.min(cnt[...,1]), np.ndarray.max(cnt[...,1])
            
    return x_min, x_max, y_min, y_max
    

def get_contours(img):
    contours, _= cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    all_contour = img.copy()
    
    cnt = sorted(contours, key=cv2.contourArea, reverse=True)[0]
    
    one_contour = cv2.drawContours(img.copy(), cnt, -1, (0, 255, 0), 2)
    
    for certain_contour in contours:
        
        rect = cv2.minAreaRect(certain_contour)
        box = cv2.boxPoints(rect)
        box = np.intp(box)
        
        all_contour = cv2.drawContours(all_contour, [box], -1, (0, 255, 0), 2)
        
    return all_contour, one_contour, cnt

In [3]:
'''
 하나의 이미지 Visualization
'''
# origin_img = cv2.imread('01597119_F_59_2022-05-28_0_1.jpg', cv2.IMREAD_GRAYSCALE)
# origin_img = cv2.imread('00607925_F_60_2022-10-24_21_1.jpg', cv2.IMREAD_GRAYSCALE)
origin_img = cv2.imread('00796540_F_69_2020-07-27_17_1.jpg', cv2.IMREAD_GRAYSCALE)
# origin_img = cv2.imread('00734411_F_55_2022-02-25_115_1.jpg', cv2.IMREAD_GRAYSCALE)

## 예외 CASE
# origin_img = cv2.imread('01403512_M_78_2022-02-07_0_1.jpg', cv2.IMREAD_GRAYSCALE)

img = origin_img.copy()

canny_img = canny(img, 70, 150)
all_contour, one_contour, contours = get_contours(canny_img.copy())
x_min, x_max, y_min, y_max= find_min_max(contours)

sliced_img = origin_img[y_min-Margin:y_max+Margin,x_min-Margin:x_max+Margin]

print("size of sliced image: ", sliced_img.shape)

resize_img = cv2.resize(sliced_img, (448, 224))
resize_img_invert = cv2.bitwise_not(resize_img)
ret, binary_img = cv2.threshold(resize_img_invert, 50, 255, cv2.THRESH_BINARY)


cv2.imshow('origin_img', origin_img)
cv2.imshow('canny_img', canny_img)
cv2.imshow('all_contour', all_contour)
cv2.imshow('one_contour', one_contour)
cv2.imshow('sliced_img', sliced_img)
# cv2.imshow('resize_img', resize_img)
# cv2.imshow('sliced_ivnert', resize_img_invert)
cv2.imshow('binary_img', binary_img)

save_file_name = "./binary_img.jpg"
save_file_name2 = "./sliced.jpg"
cv2.imwrite(save_file_name,binary_img)
cv2.imwrite(save_file_name2,sliced_img)

cv2.waitKey(0)
cv2.destroyAllWindows()

size of sliced image:  (220, 452)


In [2]:
def crop_resizing_binary(origin_img):
    img = origin_img.copy()
    canny_img = canny(img, 70, 150)
    all_contour, one_contour, contours = get_contours(canny_img.copy())
    x_min, x_max, y_min, y_max= find_min_max(contours)

    sliced_img = origin_img[y_min-Margin:y_max+Margin,x_min-Margin:x_max+Margin]

    if sliced_img.shape[0] < 200 or sliced_img.shape[1] < 400:
        print("Exception :" , sliced_img.shape)
        return sliced_img, 0
    else:
        resize_img = cv2.resize(sliced_img, (448, 224))
        resize_img_invert = cv2.bitwise_not(resize_img)
        ret, binary_img = cv2.threshold(resize_img_invert, 50, 255, cv2.THRESH_BINARY)
        return binary_img, 1

In [3]:
def resizing_binary(origin_img):
    img = origin_img.copy()
    resize_img = cv2.resize(img, (448, 224))
    resize_img_invert = cv2.bitwise_not(resize_img)
    ret, binary_img = cv2.threshold(resize_img_invert, 50, 255, cv2.THRESH_BINARY)
    return binary_img, 1

In [6]:
red_dir = 'new_data/red'
yellow_dir = 'new_data/yellow'
green_dir = 'new_data/green'

red_list = natsort.natsorted(os.listdir(red_dir))
yellow_list = natsort.natsorted(os.listdir(yellow_dir))
green_list = natsort.natsorted(os.listdir(green_dir))

save_dir_red = 'new_data_preprocessed/red'
save_dir_yellow = 'new_data_preprocessed/yellow'
save_dir_green = 'new_data_preprocessed/green'

for i in range(len(red_list)):
    origin_img_name = red_list[i]
    # print(os.path.join(red_dir, origin_img_name))
    origin_img = cv2.imread(os.path.join(red_dir, origin_img_name), cv2.IMREAD_GRAYSCALE)
    
    processed_img, save_point = crop_resizing_binary(origin_img)
    if save_point:
        cv2.imwrite(os.path.join(save_dir_red, origin_img_name), processed_img)

for i in range(len(yellow_list)):
    origin_img_name = yellow_list[i]
    origin_img = cv2.imread(os.path.join(yellow_dir, origin_img_name), cv2.IMREAD_GRAYSCALE)
    
    processed_img, save_point = crop_resizing_binary(origin_img)
    if save_point:
        cv2.imwrite(os.path.join(save_dir_yellow, origin_img_name), processed_img)    

for i in range(len(green_list)):
    origin_img_name = green_list[i]
    origin_img = cv2.imread(os.path.join(green_dir, origin_img_name), cv2.IMREAD_GRAYSCALE)
    
    processed_img, save_point = crop_resizing_binary(origin_img)
    if save_point:
        cv2.imwrite(os.path.join(save_dir_green, origin_img_name), processed_img)


In [ ]:
red_dir = 'new_data/red'
yellow_dir = 'new_data/yellow'
green_dir = 'new_data/green'

red_list = natsort.natsorted(os.listdir(red_dir))
yellow_list = natsort.natsorted(os.listdir(yellow_dir))
green_list = natsort.natsorted(os.listdir(green_dir))

save_dir_red = 'new_data_preprocessed/red'
save_dir_yellow = 'new_data_preprocessed/yellow'
save_dir_green = 'new_data_preprocessed/green'

for i in range(len(red_list)):
    origin_img_name = red_list[i]
    # print(os.path.join(red_dir, origin_img_name))
    origin_img = cv2.imread(os.path.join(red_dir, origin_img_name), cv2.IMREAD_GRAYSCALE)
    
    processed_img, save_point = crop_resizing_binary(origin_img)
    if save_point:
        cv2.imwrite(os.path.join(save_dir_red, origin_img_name), processed_img)

for i in range(len(yellow_list)):
    origin_img_name = yellow_list[i]
    origin_img = cv2.imread(os.path.join(yellow_dir, origin_img_name), cv2.IMREAD_GRAYSCALE)
    
    processed_img, save_point = crop_resizing_binary(origin_img)
    if save_point:
        cv2.imwrite(os.path.join(save_dir_yellow, origin_img_name), processed_img)    

for i in range(len(green_list)):
    origin_img_name = green_list[i]
    origin_img = cv2.imread(os.path.join(green_dir, origin_img_name), cv2.IMREAD_GRAYSCALE)
    
    processed_img, save_point = crop_resizing_binary(origin_img)
    if save_point:
        cv2.imwrite(os.path.join(save_dir_green, origin_img_name), processed_img)


In [6]:
# processing aur, bph, normal
aur_dir = 'crop_image/aur'
bph_dir = 'crop_image/bph'
normal_dir = 'crop_image/normal'

aur_list = natsort.natsorted(os.listdir(aur_dir))
bph_list = natsort.natsorted(os.listdir(bph_dir))
normal_list = natsort.natsorted(os.listdir(normal_dir))

save_dir_aur = 'crop_processed/aur'
save_dir_bph = 'crop_processed/bph'
save_dir_normal = 'crop_processed/normal'

for i in range(len(aur_list)):
    origin_img_name = aur_list[i]
    # print(os.path.join(aur_dir, origin_img_name))
    origin_img = cv2.imread(os.path.join(aur_dir, origin_img_name), cv2.IMREAD_GRAYSCALE)
    
    processed_img, save_point = resizing_binary(origin_img)
    if save_point:
        cv2.imwrite(os.path.join(save_dir_aur, origin_img_name), processed_img)

for i in range(len(bph_list)):
    origin_img_name = bph_list[i]
    origin_img = cv2.imread(os.path.join(bph_dir, origin_img_name), cv2.IMREAD_GRAYSCALE)
    
    processed_img, save_point = resizing_binary(origin_img)
    if save_point:
        cv2.imwrite(os.path.join(save_dir_bph, origin_img_name), processed_img)    

for i in range(len(normal_list)):
    origin_img_name = normal_list[i]
    origin_img = cv2.imread(os.path.join(normal_dir, origin_img_name), cv2.IMREAD_GRAYSCALE)
    
    processed_img, save_point = resizing_binary(origin_img)
    if save_point:
        cv2.imwrite(os.path.join(save_dir_normal, origin_img_name), processed_img)


In [7]:
# processing oab, stricture, ub
oab_dir = 'crop_image/oab'
stricture_dir = 'crop_image/stricture'
ub_dir = 'crop_image/ub'

oab_list = natsort.natsorted(os.listdir(oab_dir))
stricture_list = natsort.natsorted(os.listdir(stricture_dir))
ub_list = natsort.natsorted(os.listdir(ub_dir))

save_dir_oab = 'crop_processed/oab'
save_dir_stricture = 'crop_processed/stricture'
save_dir_ub = 'crop_processed/ub'

for i in range(len(oab_list)):
    origin_img_name = oab_list[i]
    # print(os.path.join(oab_dir, origin_img_name))
    origin_img = cv2.imread(os.path.join(oab_dir, origin_img_name), cv2.IMREAD_GRAYSCALE)
    
    processed_img, save_point = resizing_binary(origin_img)
    if save_point:
        cv2.imwrite(os.path.join(save_dir_oab, origin_img_name), processed_img)

for i in range(len(stricture_list)):
    origin_img_name = stricture_list[i]
    origin_img = cv2.imread(os.path.join(stricture_dir, origin_img_name), cv2.IMREAD_GRAYSCALE)
    
    processed_img, save_point = resizing_binary(origin_img)
    if save_point:
        cv2.imwrite(os.path.join(save_dir_stricture, origin_img_name), processed_img)    

for i in range(len(ub_list)):
    origin_img_name = ub_list[i]
    origin_img = cv2.imread(os.path.join(ub_dir, origin_img_name), cv2.IMREAD_GRAYSCALE)
    
    processed_img, save_point = resizing_binary(origin_img)
    if save_point:
        cv2.imwrite(os.path.join(save_dir_ub, origin_img_name), processed_img)


In [5]:
# processing new_data_origin to processed aur, bph, normal
aur_dir = 'new_data_origin/AUR_new'
bph_dir = 'new_data_origin/BPH_new'
normal_dir = 'new_data_origin/Normal_new'

aur_list = natsort.natsorted(os.listdir(aur_dir))
bph_list = natsort.natsorted(os.listdir(bph_dir))
normal_list = natsort.natsorted(os.listdir(normal_dir))

save_dir_aur = 'origin_processed/aur'
save_dir_bph = 'origin_processed/bph'
save_dir_normal = 'origin_processed/normal'

for i in range(len(aur_list)):
    origin_img_name = aur_list[i]
    origin_img = cv2.imread(os.path.join(aur_dir, origin_img_name), cv2.IMREAD_GRAYSCALE)
    
    processed_img, save_point = crop_resizing_binary(origin_img)
    if save_point:
        cv2.imwrite(os.path.join(save_dir_aur, origin_img_name), processed_img)

for i in range(len(bph_list)):
    origin_img_name = bph_list[i]
    origin_img = cv2.imread(os.path.join(bph_dir, origin_img_name), cv2.IMREAD_GRAYSCALE)
    
    processed_img, save_point = crop_resizing_binary(origin_img)
    if save_point:
        cv2.imwrite(os.path.join(save_dir_bph, origin_img_name), processed_img)    

for i in range(len(normal_list)):
    origin_img_name = normal_list[i]
    origin_img = cv2.imread(os.path.join(normal_dir, origin_img_name), cv2.IMREAD_GRAYSCALE)
    
    processed_img, save_point = crop_resizing_binary(origin_img)
    if save_point:
        cv2.imwrite(os.path.join(save_dir_normal, origin_img_name), processed_img)


Exception : (67, 434)
Exception : (214, 371)
Exception : (138, 80)
Exception : (320, 309)
Exception : (221, 375)
Exception : (150, 247)
Exception : (185, 438)
Exception : (222, 392)
Exception : (293, 319)
Exception : (111, 144)
Exception : (218, 370)
Exception : (190, 430)
Exception : (221, 372)
Exception : (217, 279)
Exception : (220, 370)
Exception : (222, 286)
Exception : (131, 398)
Exception : (226, 385)
Exception : (222, 398)
Exception : (123, 146)
Exception : (227, 351)
Exception : (217, 335)
Exception : (37, 443)
Exception : (219, 268)
Exception : (215, 350)
Exception : (222, 284)
Exception : (150, 315)
Exception : (225, 314)
Exception : (228, 327)
Exception : (93, 512)
Exception : (208, 266)
Exception : (226, 308)
Exception : (37, 443)
Exception : (222, 339)
Exception : (220, 391)
Exception : (220, 386)
Exception : (111, 143)
Exception : (233, 354)
Exception : (221, 396)
Exception : (227, 312)
Exception : (221, 370)
Exception : (218, 347)
Exception : (220, 333)
Exception : (232

In [10]:
# processing new_data_origin to processed OAB, Stricture, UB
OAB_new = 'new_data_origin/OAB_new/OAB'
Stricture_new = 'new_data_origin/Stricture_new'
Ub = 'new_data_origin/Underactive bladder_new'

oab_list = natsort.natsorted(os.listdir(OAB_new))
stricture_list = natsort.natsorted(os.listdir(Stricture_new))
ub_list = natsort.natsorted(os.listdir(Ub))

save_dir_oab = 'origin_processed/oab'
save_dir_stricture = 'origin_processed/stricture'
save_dir_ub = 'origin_processed/ub'


for i in range(len(oab_list)):
    origin_img_name = oab_list[i]
    origin_img = cv2.imread(os.path.join(OAB_new, origin_img_name), cv2.IMREAD_GRAYSCALE)
    
    processed_img, save_point = crop_resizing_binary(origin_img)
    if save_point:
        cv2.imwrite(os.path.join(save_dir_oab, origin_img_name), processed_img)

for i in range(len(stricture_list)):
    origin_img_name = stricture_list[i]
    origin_img = cv2.imread(os.path.join(Stricture_new, origin_img_name), cv2.IMREAD_GRAYSCALE)
    
    processed_img, save_point = crop_resizing_binary(origin_img)
    if save_point:
        cv2.imwrite(os.path.join(save_dir_stricture, origin_img_name), processed_img)    

for i in range(len(ub_list)):
    origin_img_name = ub_list[i]
    origin_img = cv2.imread(os.path.join(Ub, origin_img_name), cv2.IMREAD_GRAYSCALE)
    
    processed_img, save_point = crop_resizing_binary(origin_img)
    if save_point:
        cv2.imwrite(os.path.join(save_dir_ub, origin_img_name), processed_img)


Exception : (298, 237)
Exception : (226, 384)
Exception : (219, 357)
Exception : (132, 431)
Exception : (212, 279)
Exception : (226, 376)
Exception : (220, 386)
Exception : (79, 434)
Exception : (220, 330)
Exception : (64, 434)
Exception : (153, 447)
Exception : (226, 397)
Exception : (220, 398)
Exception : (222, 335)
Exception : (222, 364)
Exception : (220, 396)
Exception : (222, 349)
Exception : (227, 397)
Exception : (221, 383)
Exception : (218, 232)
Exception : (222, 398)
Exception : (194, 394)
Exception : (46, 434)
Exception : (227, 370)
Exception : (234, 388)
Exception : (217, 307)
Exception : (52, 387)
Exception : (225, 375)
Exception : (219, 279)
Exception : (220, 385)
Exception : (187, 439)
Exception : (189, 390)
Exception : (179, 323)
Exception : (219, 376)
Exception : (105, 125)
Exception : (232, 267)
Exception : (220, 341)
Exception : (223, 174)
Exception : (57, 434)
Exception : (226, 398)
Exception : (231, 217)
Exception : (214, 342)
Exception : (111, 161)
Exception : (48,